# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [ ]:
import os

def busqueda_lineal_documentos(carpeta, palabra_buscar):
    # Obtener la lista de archivos en la carpeta
    documentos = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.txt')]
    
    documentos_con_palabra = []

    # Recorrer cada documento
    for doc in documentos:
        ruta_doc = os.path.join(carpeta, doc)
        with open(ruta_doc, 'r', encoding='utf-8') as f:
            contenido = f.read()
            if palabra_buscar.lower() in contenido.lower():
                documentos_con_palabra.append(doc)

    if documentos_con_palabra:
        print(f"La palabra '{palabra_buscar}' aparece en los siguientes documentos:")
        for doc in documentos_con_palabra:
            print(f"- {doc}")
    else:
        print(f"La palabra '{palabra_buscar}' no se encontró en ningún documento.")

# Ejemplo de uso
if __name__ == "__main__":
    carpeta = 'C:\Users\fricM\OneDrive\Desktop\Recuperación de la Información\data'  # Asegúrate de tener tu carpeta de documentos aquí
    palabra = input("Ingresa la palabra a buscar: ")
    busqueda_lineal_documentos(carpeta, palabra)


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_